# 机器学习 决策树

### 1.数据集选择

使用UCI提供的车辆评价数据集进行决策树构建。

数据集源地址：https://archive.ics.uci.edu/ml/datasets/Car+Evaluation

数据集包含4个类别，6条属性，共1728条数据，属性及取值如下：

buying|maint|doors|persons|lug_boot|safety
------|-----|-----|-------|--------|------
vhigh|vhigh|2|2|small|low
high|high|3|4|med|med|
med|med|4|more|big|high
low|low|5more| | 

### 2.决策树的Python实现

程序实现了ID3、C4.5以及CART三种决策树构建算法，运行时根据参数选择相应的构建算法并构建决策树。

对于ID3，计算信息熵增益作为划分指标。

对于C4.5，计算信息熵增益率作为划分指标。

对于CART，计算基尼指数作为划分指标。

### ID3决策树生成：

In [13]:
import pandas as pd
import math
import numpy as np
import time
import imp

algorithm = "ID3" #ID3, C4.5, CART

num_of_trees = 3
dump_to_console = True
epochs = 10
learning_rate = 1
df = pd.read_csv("D:\\Env\\vscode\\DeepLearning\\decision-trees-for-ml\\dataset\car.data",names=["buying","maint","doors","persons","lug_boot","safety","Decision"])
dataset = df.copy()

if df['Decision'].dtypes != 'object':
	algorithm = 'Regression'
	global_stdev = df['Decision'].std(ddof=0)

print(algorithm," tree is going to be built...")

dataset_features = dict() 

def softmax(w):
	e = np.exp(np.array(w))
	dist = e / np.sum(e)
	return dist

def sign(x):
	if x > 0:
		return 1
	elif x < 0:
		return -1
	else:
		return 0
	
def processContinuousFeatures(df, column_name, entropy):
	unique_values = sorted(df[column_name].unique())
	#print(column_name,"->",unique_values)
	
	subset_gainratios = []; subset_gains = []; subset_ginis = []; subset_red_stdevs = []
	
	for i in range(0, len(unique_values)-1):
		threshold = unique_values[i]
		
		subset1 = df[df[column_name] <= threshold]
		subset2 = df[df[column_name] > threshold]
		
		subset1_rows = subset1.shape[0]; subset2_rows = subset2.shape[0]
		total_instances = df.shape[0]
		
		subset1_probability = subset1_rows / total_instances
		subset2_probability = subset2_rows / total_instances
		
		if algorithm == 'ID3' or algorithm == 'C4.5':
			threshold_gain = entropy - subset1_probability*calculateEntropy(subset1) - subset2_probability*calculateEntropy(subset2)
			subset_gains.append(threshold_gain)
		
		if algorithm == 'C4.5':
			threshold_splitinfo = -subset1_probability * math.log(subset1_probability, 2)-subset2_probability*math.log(subset2_probability, 2)
			gainratio = threshold_gain / threshold_splitinfo
			subset_gainratios.append(gainratio)
				
		elif algorithm == 'CART':
			decision_for_subset1 = subset1['Decision'].value_counts().tolist()
			decision_for_subset2 = subset2['Decision'].value_counts().tolist()
			
			gini_subset1 = 1; gini_subset2 = 1
			
			for j in range(0, len(decision_for_subset1)):
				gini_subset1 = gini_subset1 - math.pow((decision_for_subset1[j]/subset1_rows),2)
			
			for j in range(0, len(decision_for_subset2)):
				gini_subset2 = gini_subset2 - math.pow((decision_for_subset2[j]/subset2_rows),2)
			
			gini = (subset1_rows/total_instances)*gini_subset1 + (subset2_rows/total_instances) * gini_subset2
			
			subset_ginis.append(gini)
	
	if algorithm == "C4.5":
		winner_one = subset_gainratios.index(max(subset_gainratios))
	elif algorithm == "ID3":
		winner_one = subset_gains.index(max(subset_gains))
	elif algorithm == "CART":
		winner_one = subset_ginis.index(min(subset_ginis))
		
	winner_threshold = unique_values[winner_one]
	
	df[column_name] = np.where(df[column_name] <= winner_threshold, "<="+str(winner_threshold), ">"+str(winner_threshold))
	
	return df

def calculateEntropy(df):
	
	if algorithm == 'Regression':
		return 0
	
	instances = df.shape[0]; columns = df.shape[1]

	decisions = df['Decision'].value_counts().keys().tolist()

	entropy = 0

	for i in range(0, len(decisions)):
		decision = decisions[i]
		num_of_decisions = df['Decision'].value_counts().tolist()[i]
		
		class_probability = num_of_decisions/instances
		
		entropy = entropy - class_probability*math.log(class_probability, 2)
		
	return entropy

def findDecision(df):
		
	entropy = calculateEntropy(df)
	#print("entropy: ",entropy)

	columns = df.shape[1]; instances = df.shape[0]

	gains = []; gainratios = []; ginis = []; reducted_stdevs = []

	for i in range(0, columns-1):
		column_name = df.columns[i]
		column_type = df[column_name].dtypes
		
		
		if column_type != 'object':
			df = processContinuousFeatures(df, column_name, entropy)
		
		classes = df[column_name].value_counts()
		
		gain = entropy * 1; splitinfo = 0; gini = 0; weighted_stdev = 0
		
		for j in range(0, len(classes)):
			current_class = classes.keys().tolist()[j]
			#print(column_name,"->",current_class)
			
			subdataset = df[df[column_name] == current_class]
			#print(subdataset)
			
			subset_instances = subdataset.shape[0]
			class_probability = subset_instances/instances
			
			if algorithm == 'ID3' or algorithm == 'C4.5':
				subset_entropy = calculateEntropy(subdataset)
				#print("entropy for this sub dataset is ", subset_entropy)
				gain = gain - class_probability * subset_entropy			
			
			if algorithm == 'C4.5':
				splitinfo = splitinfo - class_probability*math.log(class_probability, 2)
			
			elif algorithm == 'CART': #GINI index
				decision_list = subdataset['Decision'].value_counts().tolist()
				
				subgini = 1
				
				for k in range(0, len(decision_list)):
					subgini = subgini - math.pow((decision_list[k]/subset_instances), 2)
				
				gini = gini + (subset_instances / instances) * subgini
			
			elif algorithm == 'Regression':
				subset_stdev = subdataset['Decision'].std(ddof=0)
				weighted_stdev = weighted_stdev + (subset_instances/instances)*subset_stdev
		
		
		if algorithm == "ID3":
			gains.append(gain)
		
		elif algorithm == "C4.5":
		
			if splitinfo == 0:
				splitinfo = 100 
				
			gainratio = gain / splitinfo
			gainratios.append(gainratio)
		
		elif algorithm == "CART":
			ginis.append(gini)
	
	#print(df)
	if algorithm == "ID3":
		winner_index = gains.index(max(gains))
	elif algorithm == "C4.5":
		winner_index = gainratios.index(max(gainratios))
	elif algorithm == "CART":
		winner_index = ginis.index(min(ginis))
	winner_name = df.columns[winner_index]

	return winner_name
	
def formatRule(root):
	resp = ''
	
	for i in range(0, root):
		resp = resp + '   '
	
	return resp	

def storeRule(file,content):
		f = open(file, "a+")
		f.writelines(content)
		f.writelines("\n")

def createFile(file,content):
		f = open(file, "w")
		f.write(content)
	
def buildDecisionTree(df,root,file):
	#print(df.shape)
	charForResp = "'"
	if algorithm == 'Regression':
		charForResp = ""

	tmp_root = root * 1
	
	df_copy = df.copy()
	
	winner_name = findDecision(df)
	
	j = 0 
	for i in dataset_features:
		if i == winner_name:
			winner_index = j
		j = j + 1
	
	numericColumn = False
	if dataset_features[winner_name] != 'object':
		numericColumn = True
	
	columns = df.shape[1]
	for i in range(0, columns-1):
		column_name = df.columns[i]; column_type = df[column_name].dtypes
		if column_type != 'object' and column_name != winner_name:
			df[column_name] = df_copy[column_name]
	
	classes = df[winner_name].value_counts().keys().tolist()

	for i in range(0,len(classes)):
		current_class = classes[i]
		subdataset = df[df[winner_name] == current_class]
		subdataset = subdataset.drop(columns=[winner_name])
		
		if numericColumn == True:
			compareTo = current_class
		else:
			compareTo = " == '"+str(current_class)+"'"
		
		
		terminateBuilding = False

		if len(subdataset['Decision'].value_counts().tolist()) == 1:
			final_decision = subdataset['Decision'].value_counts().keys().tolist()[0] #all items are equal in this case
			terminateBuilding = True
		elif subdataset.shape[1] == 1: 
			final_decision = subdataset['Decision'].value_counts().idxmax() #get the most frequent one
			terminateBuilding = True
		
		if dump_to_console == True:
			print(formatRule(root),"if ",winner_name,compareTo,":")
		else:
			#storeRule(file,(formatRule(root),"if ",winner_name,compareTo,":"))
			storeRule(file,(formatRule(root),"if obj[",str(winner_index),"]",compareTo,":"))
		
		
		if terminateBuilding == True: #check decision is made
			if dump_to_console == True:
				print(formatRule(root+1),"return ",charForResp+str(final_decision)+charForResp)
			else:
				storeRule(file,(formatRule(root+1),"return ",charForResp+str(final_decision)+charForResp))
		else: #decision is not made, continue to create branch and leafs
			root = root + 1 #the following rule will be included by this rule. increase root
			buildDecisionTree(subdataset,root,file)
		
		root = tmp_root * 1
		

if(True): 
	header = "def findDecision("
	num_of_columns = df.shape[1]-1
	for i in range(0, num_of_columns):
		if dump_to_console == True:
			if i > 0:
				header = header + ","
			header = header + df.columns[i]
		
		column_name = df.columns[i]
		dataset_features[column_name] = df[column_name].dtypes

	if dump_to_console == False:
		header = header + "obj"
		
	header = header + "):\n"

	if dump_to_console == True:
		print(header,end='')

begin = time.time()

for i in range(0, num_of_trees):
	subset = df.sample(frac=1/num_of_trees)
	
	root = 1
	
	file = "rule_"+str(i)+".py"
	
	if dump_to_console == False:
		createFile(file, header)
	
	buildDecisionTree(subset,root, file)

print("finished in ",time.time() - begin," seconds")

ID3  tree is going to be built...
def findDecision(buying,maint,doors,persons,lug_boot,safety):
    if  safety  == 'high' :
       if  persons  == '4' :
          if  buying  == 'med' :
             if  maint  == 'low' :
                if  lug_boot  == 'big' :
                   return  'vgood'
                if  lug_boot  == 'med' :
                   if  doors  == '5more' :
                      return  'vgood'
                   if  doors  == '2' :
                      return  'good'
                   if  doors  == '3' :
                      return  'good'
                if  lug_boot  == 'small' :
                   return  'good'
             if  maint  == 'high' :
                return  'acc'
             if  maint  == 'med' :
                if  doors  == '3' :
                   if  lug_boot  == 'big' :
                      return  'vgood'
                   if  lug_boot  == 'med' :
                      return  'acc'
                if  doors  == '4' :
                 

### CART决策树生成：

In [14]:
import pandas as pd
import math
import numpy as np
import time
import imp

algorithm = "CART" #ID3, C4.5, CART

num_of_trees = 3
dump_to_console = True
epochs = 10
learning_rate = 1
df = pd.read_csv("D:\\Env\\vscode\\DeepLearning\\decision-trees-for-ml\\dataset\car.data",names=["buying","maint","doors","persons","lug_boot","safety","Decision"])
dataset = df.copy()

if df['Decision'].dtypes != 'object':
	algorithm = 'Regression'
	global_stdev = df['Decision'].std(ddof=0)

print(algorithm," tree is going to be built...")

dataset_features = dict() 

def softmax(w):
	e = np.exp(np.array(w))
	dist = e / np.sum(e)
	return dist

def sign(x):
	if x > 0:
		return 1
	elif x < 0:
		return -1
	else:
		return 0
	
def processContinuousFeatures(df, column_name, entropy):
	unique_values = sorted(df[column_name].unique())
	#print(column_name,"->",unique_values)
	
	subset_gainratios = []; subset_gains = []; subset_ginis = []; subset_red_stdevs = []
	
	for i in range(0, len(unique_values)-1):
		threshold = unique_values[i]
		
		subset1 = df[df[column_name] <= threshold]
		subset2 = df[df[column_name] > threshold]
		
		subset1_rows = subset1.shape[0]; subset2_rows = subset2.shape[0]
		total_instances = df.shape[0]
		
		subset1_probability = subset1_rows / total_instances
		subset2_probability = subset2_rows / total_instances
		
		if algorithm == 'ID3' or algorithm == 'C4.5':
			threshold_gain = entropy - subset1_probability*calculateEntropy(subset1) - subset2_probability*calculateEntropy(subset2)
			subset_gains.append(threshold_gain)
		
		if algorithm == 'C4.5':
			threshold_splitinfo = -subset1_probability * math.log(subset1_probability, 2)-subset2_probability*math.log(subset2_probability, 2)
			gainratio = threshold_gain / threshold_splitinfo
			subset_gainratios.append(gainratio)
				
		elif algorithm == 'CART':
			decision_for_subset1 = subset1['Decision'].value_counts().tolist()
			decision_for_subset2 = subset2['Decision'].value_counts().tolist()
			
			gini_subset1 = 1; gini_subset2 = 1
			
			for j in range(0, len(decision_for_subset1)):
				gini_subset1 = gini_subset1 - math.pow((decision_for_subset1[j]/subset1_rows),2)
			
			for j in range(0, len(decision_for_subset2)):
				gini_subset2 = gini_subset2 - math.pow((decision_for_subset2[j]/subset2_rows),2)
			
			gini = (subset1_rows/total_instances)*gini_subset1 + (subset2_rows/total_instances) * gini_subset2
			
			subset_ginis.append(gini)
	
	if algorithm == "C4.5":
		winner_one = subset_gainratios.index(max(subset_gainratios))
	elif algorithm == "ID3":
		winner_one = subset_gains.index(max(subset_gains))
	elif algorithm == "CART":
		winner_one = subset_ginis.index(min(subset_ginis))
		
	winner_threshold = unique_values[winner_one]
	
	df[column_name] = np.where(df[column_name] <= winner_threshold, "<="+str(winner_threshold), ">"+str(winner_threshold))
	
	return df

def calculateEntropy(df):
	
	if algorithm == 'Regression':
		return 0
	
	instances = df.shape[0]; columns = df.shape[1]

	decisions = df['Decision'].value_counts().keys().tolist()

	entropy = 0

	for i in range(0, len(decisions)):
		decision = decisions[i]
		num_of_decisions = df['Decision'].value_counts().tolist()[i]
		
		class_probability = num_of_decisions/instances
		
		entropy = entropy - class_probability*math.log(class_probability, 2)
		
	return entropy

def findDecision(df):
		
	entropy = calculateEntropy(df)
	#print("entropy: ",entropy)

	columns = df.shape[1]; instances = df.shape[0]

	gains = []; gainratios = []; ginis = []; reducted_stdevs = []

	for i in range(0, columns-1):
		column_name = df.columns[i]
		column_type = df[column_name].dtypes
		
		
		if column_type != 'object':
			df = processContinuousFeatures(df, column_name, entropy)
		
		classes = df[column_name].value_counts()
		
		gain = entropy * 1; splitinfo = 0; gini = 0; weighted_stdev = 0
		
		for j in range(0, len(classes)):
			current_class = classes.keys().tolist()[j]
			#print(column_name,"->",current_class)
			
			subdataset = df[df[column_name] == current_class]
			#print(subdataset)
			
			subset_instances = subdataset.shape[0]
			class_probability = subset_instances/instances
			
			if algorithm == 'ID3' or algorithm == 'C4.5':
				subset_entropy = calculateEntropy(subdataset)
				#print("entropy for this sub dataset is ", subset_entropy)
				gain = gain - class_probability * subset_entropy			
			
			if algorithm == 'C4.5':
				splitinfo = splitinfo - class_probability*math.log(class_probability, 2)
			
			elif algorithm == 'CART': #GINI index
				decision_list = subdataset['Decision'].value_counts().tolist()
				
				subgini = 1
				
				for k in range(0, len(decision_list)):
					subgini = subgini - math.pow((decision_list[k]/subset_instances), 2)
				
				gini = gini + (subset_instances / instances) * subgini
			
			elif algorithm == 'Regression':
				subset_stdev = subdataset['Decision'].std(ddof=0)
				weighted_stdev = weighted_stdev + (subset_instances/instances)*subset_stdev
		
		
		if algorithm == "ID3":
			gains.append(gain)
		
		elif algorithm == "C4.5":
		
			if splitinfo == 0:
				splitinfo = 100 
				
			gainratio = gain / splitinfo
			gainratios.append(gainratio)
		
		elif algorithm == "CART":
			ginis.append(gini)
	
	#print(df)
	if algorithm == "ID3":
		winner_index = gains.index(max(gains))
	elif algorithm == "C4.5":
		winner_index = gainratios.index(max(gainratios))
	elif algorithm == "CART":
		winner_index = ginis.index(min(ginis))
	winner_name = df.columns[winner_index]

	return winner_name
	
def formatRule(root):
	resp = ''
	
	for i in range(0, root):
		resp = resp + '   '
	
	return resp	

def storeRule(file,content):
		f = open(file, "a+")
		f.writelines(content)
		f.writelines("\n")

def createFile(file,content):
		f = open(file, "w")
		f.write(content)
	
def buildDecisionTree(df,root,file):
	#print(df.shape)
	charForResp = "'"
	if algorithm == 'Regression':
		charForResp = ""

	tmp_root = root * 1
	
	df_copy = df.copy()
	
	winner_name = findDecision(df)
	
	j = 0 
	for i in dataset_features:
		if i == winner_name:
			winner_index = j
		j = j + 1
	
	numericColumn = False
	if dataset_features[winner_name] != 'object':
		numericColumn = True
	
	columns = df.shape[1]
	for i in range(0, columns-1):
		column_name = df.columns[i]; column_type = df[column_name].dtypes
		if column_type != 'object' and column_name != winner_name:
			df[column_name] = df_copy[column_name]
	
	classes = df[winner_name].value_counts().keys().tolist()

	for i in range(0,len(classes)):
		current_class = classes[i]
		subdataset = df[df[winner_name] == current_class]
		subdataset = subdataset.drop(columns=[winner_name])
		
		if numericColumn == True:
			compareTo = current_class
		else:
			compareTo = " == '"+str(current_class)+"'"
		
		
		terminateBuilding = False

		if len(subdataset['Decision'].value_counts().tolist()) == 1:
			final_decision = subdataset['Decision'].value_counts().keys().tolist()[0] #all items are equal in this case
			terminateBuilding = True
		elif subdataset.shape[1] == 1: 
			final_decision = subdataset['Decision'].value_counts().idxmax() #get the most frequent one
			terminateBuilding = True
		
		if dump_to_console == True:
			print(formatRule(root),"if ",winner_name,compareTo,":")
		else:
			#storeRule(file,(formatRule(root),"if ",winner_name,compareTo,":"))
			storeRule(file,(formatRule(root),"if obj[",str(winner_index),"]",compareTo,":"))
		
		
		if terminateBuilding == True: #check decision is made
			if dump_to_console == True:
				print(formatRule(root+1),"return ",charForResp+str(final_decision)+charForResp)
			else:
				storeRule(file,(formatRule(root+1),"return ",charForResp+str(final_decision)+charForResp))
		else: #decision is not made, continue to create branch and leafs
			root = root + 1 #the following rule will be included by this rule. increase root
			buildDecisionTree(subdataset,root,file)
		
		root = tmp_root * 1
		

if(True): 
	header = "def findDecision("
	num_of_columns = df.shape[1]-1
	for i in range(0, num_of_columns):
		if dump_to_console == True:
			if i > 0:
				header = header + ","
			header = header + df.columns[i]
		
		column_name = df.columns[i]
		dataset_features[column_name] = df[column_name].dtypes

	if dump_to_console == False:
		header = header + "obj"
		
	header = header + "):\n"

	if dump_to_console == True:
		print(header,end='')

begin = time.time()

for i in range(0, num_of_trees):
	subset = df.sample(frac=1/num_of_trees)
	
	root = 1
	
	file = "rule_"+str(i)+".py"
	
	if dump_to_console == False:
		createFile(file, header)
	
	buildDecisionTree(subset,root, file)

print("finished in ",time.time() - begin," seconds")

CART  tree is going to be built...
def findDecision(buying,maint,doors,persons,lug_boot,safety):
    if  persons  == '2' :
       return  'unacc'
    if  persons  == 'more' :
       if  safety  == 'low' :
          return  'unacc'
       if  safety  == 'med' :
          if  buying  == 'high' :
             if  lug_boot  == 'small' :
                return  'unacc'
             if  lug_boot  == 'big' :
                return  'acc'
             if  lug_boot  == 'med' :
                if  doors  == '3' :
                   return  'acc'
                if  doors  == '2' :
                   return  'unacc'
                if  doors  == '4' :
                   return  'unacc'
          if  buying  == 'vhigh' :
             if  maint  == 'vhigh' :
                return  'unacc'
             if  maint  == 'low' :
                if  lug_boot  == 'small' :
                   return  'unacc'
                if  lug_boot  == 'med' :
                   if  doors  == '4' :
                   